Подготовил Илья Толстихин.
TG, WA: 89246385461. Почта: Ilya-Tolstikhin@mail.ru

In [6]:
#1 - Написать код подключения к базе данных postgresql и получения данных в формате pandas DataFrame

In [7]:
pip install psycopg2 

Note: you may need to restart the kernel to use updated packages.


In [8]:
import psycopg2
import pandas as pd
import requests
import datetime
import lxml
from urllib.parse import urlencode

In [9]:
# вводим с клавиатуры данные для подключения
print('Введите название базы данных и нажмите enter: ')
bd_name = input()
print('Введите хост и нажмите enter: ')
bd_host = input() 
print('Введите имя пользователя и нажмите enter: ')
bd_user = input()
print('Введите пароль и нажмите enter: ')
bd_password = input()

Введите название базы данных и нажмите enter: 
test_db
Введите хост и нажмите enter: 
test_host
Введите имя пользователя и нажмите enter: 
test_host
Введите пароль и нажмите enter: 
12345


In [10]:
def connection_db(bd_name, bd_host, bd_user, bd_password):
    try:
        # пытаемся подключиться к базе данных
        conn = psycopg2.connect(dbname = bd_name, user = bd_user, password = bd_password, host = bd_host)
    except:
        print('Не удалось подключиться к базе данных')
        return 'Error 0'
    print('Введите текст запроса SQL, соблюдая синтаксис:')
    sql_query = input()
    df = pd.read_sql(sql_query, con=conn)
    if df.empty:
        print('Ничего не нашлось')
        return 'Error 1'
    else:
        return df

In [11]:
connection_db(bd_name, bd_host, bd_user, bd_password)

Не удалось подключиться к базе данных


'Error 0'

На данный момент у меня нет возможности продемонстрировать работу кода, но алгоритм работы следующий: вводятся данные для подключения пользователем, после чего вызывается функция для обертывания запроса в DataFrame

In [12]:
#2 - Загрузить из excel исходный файл в pandas DataFrame

Загрузим данные с яндекс диска при помощи API (можно инициализировать локально, но для запуска ноутбука удаленно так будет удобнее)

In [13]:
# data = pd.read_excel('C:/Users/Ilya-/данные для задания.xlsx')
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

path_to_file = {
    'data_test':       ['https://disk.yandex.ru/i/NXqCJHVyEVfZYg', ';']
            }
for csv_filename, csv_link in path_to_file.items():
    final_url               = base_url + urlencode(dict(public_key=csv_link[0]))
    response                = requests.get(final_url)
    download_url            = response.json()['href']
    globals()[csv_filename] = pd.read_excel(download_url)

In [14]:
data_test = data_test.rename(columns = {'Дата отчета' : 'currency', datetime.datetime(2024, 2, 7, 0, 0) : 'route_credit', \
                            'Unnamed: 2' : 'id_client', 'Unnamed: 3' : 'id_deal', 'Unnamed: 4' : 'type_deal', \
                            'Unnamed: 5' : 'id_operation', 'Unnamed: 6' : 'subsidy', 'Unnamed: 7' : 'portfolio'})
data = data_test
data_test.head(10)

,currency,route_credit,id_client,id_deal,type_deal,id_operation,subsidy,portfolio
0,Ставка ЦБ,16,NaN,NaN,NaN,NaN,NaN,NaN
1,Доллар США,90.6842,NaN,NaN,NaN,NaN,NaN,NaN
2,Евро,97.444,NaN,NaN,NaN,NaN,NaN,NaN
3,Китайский юань,12.5926,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Валюта,Направление кредитования,ID клиента,ID сделки,Тип сделки,ID фин.операции,Наличие субсидии Да/Нет,Портфель
6,RUB,Финансирование импорта,10000162059,10003403968,КредЛиния,10000000632,да,импорт
7,RUB,NaN,10000162059,10003403968,Транш,10000000633,да,нет
8,RUB,Предэкспортное финансирование,10000160314,10003404118,КредЛиния,10000000632,да,экспорт
9,RUB,NaN,10000160314,10003404118,Транш,10000000633,да,нет


In [15]:
#очищаем от шапки верхнего левого угла с данными курса валют
for i in range(data_test[data_test['currency'] == 'Валюта'].index[0] + 1):
    data_test = data_test.drop(i)
data_test.reset_index().drop(columns = ['index'])

,currency,route_credit,id_client,id_deal,type_deal,id_operation,subsidy,portfolio
0,RUB,Финансирование импорта,10000162059,10003403968,КредЛиния,10000000632,да,импорт
1,RUB,NaN,10000162059,10003403968,Транш,10000000633,да,нет
2,RUB,Предэкспортное финансирование,10000160314,10003404118,КредЛиния,10000000632,да,экспорт
3,RUB,NaN,10000160314,10003404118,Транш,10000000633,да,нет
4,RUB,Финансирование импорта,10000162507,10003404121,КредЛиния,10000000632,да,импорт
...,...,...,...,...,...,...,...,...
110,RUB,Предэкспортное финансирование,10000128603,10003440058,КредЛиния,10000000632,нет,экспорт
111,RUB,Предэкспортное финансирование,10000159538,10003441177,КредЛиния,10000000632,да,экспорт
112,RUB,Финансирование дебиторской задолженности,10000133827,10003441567,КредЛиния,10000000632,да,экспорт
113,RUB,Предэкспортное финансирование,10000160451,10003442238,КредЛиния,10000000632,нет,экспорт


In [16]:
#2.1 - Обработаться данные: Оставить только основную таблицу, В "Портфель" заменить нет на экспорт, 
#заполнить пустые данные в направлении кредитования словом "прочее"

In [17]:
data_test.portfolio = data_test.portfolio.replace('нет', 'экспорт')
data_test.route_credit = data_test.route_credit.fillna('прочее')

In [18]:
data_test

,currency,route_credit,id_client,id_deal,type_deal,id_operation,subsidy,portfolio
6,RUB,Финансирование импорта,10000162059,10003403968,КредЛиния,10000000632,да,импорт
7,RUB,прочее,10000162059,10003403968,Транш,10000000633,да,экспорт
8,RUB,Предэкспортное финансирование,10000160314,10003404118,КредЛиния,10000000632,да,экспорт
9,RUB,прочее,10000160314,10003404118,Транш,10000000633,да,экспорт
10,RUB,Финансирование импорта,10000162507,10003404121,КредЛиния,10000000632,да,импорт
...,...,...,...,...,...,...,...,...
116,RUB,Предэкспортное финансирование,10000128603,10003440058,КредЛиния,10000000632,нет,экспорт
117,RUB,Предэкспортное финансирование,10000159538,10003441177,КредЛиния,10000000632,да,экспорт
118,RUB,Финансирование дебиторской задолженности,10000133827,10003441567,КредЛиния,10000000632,да,экспорт
119,RUB,Предэкспортное финансирование,10000160451,10003442238,КредЛиния,10000000632,нет,экспорт


In [19]:
 #2.2. - Добавить колонки с 0 "Остаток лимита руб. MM-ГГГГ" и "Плановое Погашение руб. ММ-ГГГГ", 
    #где ММ-ГГГГ - каждый месяц с текущего дня по 2027 год включительно(с возможностью поставить любой другой год)

К сожалению не совсем понял суть поставленной задачи, но результат не противоречит выполнению следующего

In [20]:
mounth = datetime.datetime.now().strftime('%m-%Y')
data_test[f'balance_limit_{mounth}'] = 0
data_test[f'plan_repayment_{mounth}'] = 0

In [21]:
data_test

,currency,route_credit,id_client,id_deal,type_deal,id_operation,subsidy,portfolio,balance_limit_08-2024,plan_repayment_08-2024
6,RUB,Финансирование импорта,10000162059,10003403968,КредЛиния,10000000632,да,импорт,0,0
7,RUB,прочее,10000162059,10003403968,Транш,10000000633,да,экспорт,0,0
8,RUB,Предэкспортное финансирование,10000160314,10003404118,КредЛиния,10000000632,да,экспорт,0,0
9,RUB,прочее,10000160314,10003404118,Транш,10000000633,да,экспорт,0,0
10,RUB,Финансирование импорта,10000162507,10003404121,КредЛиния,10000000632,да,импорт,0,0
...,...,...,...,...,...,...,...,...,...,...
116,RUB,Предэкспортное финансирование,10000128603,10003440058,КредЛиния,10000000632,нет,экспорт,0,0
117,RUB,Предэкспортное финансирование,10000159538,10003441177,КредЛиния,10000000632,да,экспорт,0,0
118,RUB,Финансирование дебиторской задолженности,10000133827,10003441567,КредЛиния,10000000632,да,экспорт,0,0
119,RUB,Предэкспортное финансирование,10000160451,10003442238,КредЛиния,10000000632,нет,экспорт,0,0


In [22]:
#2.3. -  Агрегировать данные в разрезе: "Портфель", "Наличие субсидии Да/Нет" (вместе), только где "Тип Сделки" = 'КредЛиния',
    #а данные в колонках - созданные столбцы "Остаток лимита руб. MM-ГГГГ"

In [23]:
data_test_agg = data_test.query("type_deal == 'КредЛиния'") \
    .groupby(['portfolio', 'subsidy'], as_index = False) \
    .agg({f'balance_limit_{mounth}' : ['sum','mean', 'median', 'min', 'max']})
data_test_agg

portfolio subsidy balance_limit_08-2024                    
                                                   sum mean median min max
0  Прочее финансирование      да                     0  0.0    0.0   0   0
1                 импорт      да                     0  0.0    0.0   0   0
2                 импорт     нет                     0  0.0    0.0   0   0
3                экспорт      да                     0  0.0    0.0   0   0
4                экспорт     нет                     0  0.0    0.0   0   0

In [24]:
#3 - Выгрузить полученные агрегированные данные в excel на рабочий стол

In [25]:
path_to_work_table = 'C:/Users/Public/Desktop/' + 'task_3.xlsx'
data_test_agg.to_excel(f'{path_to_work_table}', sheet_name='Агрегированные данные') 

In [26]:
#4 - Получить курсы валют по текущему дню с сайта ЦБРФ: "Доллар США", "Евро", "Китайский юань"

In [48]:
date_today = datetime.datetime.now().strftime('%d/%m/%Y')
url = f'https://www.cbr.ru/scripts/XML_daily.asp?date_req={date_today}'
df = pd.read_xml(url, parser='etree', encoding='cp1251')
df.Value = df.Value.replace(',', '.')
df.query("CharCode == 'USD' or CharCode =='EUR' or CharCode == 'CNY'") \
    .groupby(['Name'], as_index = False) \
    .agg({'Value' : 'sum'})

,Name,Value
0,Доллар США,"85,1646"
1,Евро,"93,0265"
2,Китайский юань,"11,7890"
